# Customers Who Bought This Frequently Buy This! #

In [1]:
from config.config import CONFIGURATION_PATH
from matplotlib import pyplot as plt
import utils.images_utils as hlpimage
import utils.read_utils as hlpread
import utils.write_utils as hlwrite
from os.path import exists
#import pickle
import cv2
import sys
import os
import gc

In [2]:
current_dir = os.path.dirname(os.path.realpath('__file__'))
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, os.pardir))

while os.path.exists(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH)) == False:

    PROJECT_ROOT = os.path.abspath(os.path.join(PROJECT_ROOT, os.pardir))

#PROJECT_ROOT

In [3]:
TRANSACTION_DATA = os.path.join(PROJECT_ROOT, 
                                hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),'data_source','data_folders'),
                                hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),'data_source','processed_data_folder'),
                                hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),'data_source','clean_transaction_data'),
                                )
                                
tran_data = hlpread.read_from_parquet(TRANSACTION_DATA)

max_t_date = tran_data.t_dat.max()
min_t_date = tran_data.t_dat.min()

print(f'Transaction records from: {min_t_date} to {max_t_date}')
print(f'Shape of train: {tran_data.shape}')

Transaction records from: 2018-09-20 00:00:00 to 2020-09-22 00:00:00
Shape of train: (27306439, 6)


## Generate Top 30 items that are brought together with the item by other users ##

In [ ]:
#Count number of times the items where purchased
vc = tran_data.article_id.value_counts() 

print(f'Lenght of purchase count: {len(vc)}')
pairs_items = {}

vc = vc.reset_index()
vc.rename(columns = {'index': 'article_id', 'article_id': 'cnt'}, inplace = True)
vc = vc[~vc.article_id.isin(pairs_items.keys())]

for item, item_count in zip(vc.article_id, vc.cnt):

    #Only consider those items which where sold more then once
    if item_count > 1:

        if item not in pairs_items:
            
            #Get list of user who has purchase the same item
            USERS = tran_data.loc[tran_data.article_id == item, 'customer_id'].unique()

            #Get list of items those USERS have purchased with ignoring the current item
            vc2 = tran_data.loc[(tran_data.customer_id.isin(USERS)) & (tran_data.article_id != item), 'article_id'].value_counts()

            #Get Top 30 items those user has purchase based on items value counts
            other_items = []
            for i in range(0, 30):

                if i < len(vc2.index):
                    other_items.append(vc2.index[i])

                else: #Since if i reach more then the len meanse we do not have records i the vc2 list
                    break

            pairs_items[item] = other_items
            #pairs[i.item()] = [vc2.index[0], vc2.index[1], vc2.index[2]]

    else:
        # Since we know that value_count return sorted array, 
        # so if any item_count == 1 then all the other items that will come next will be equal to 1
        # So we shall break from the loop
        break

In [6]:
ITEMS_PAIRED_TOGETHER = os.path.join(PROJECT_ROOT, 
                                     hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),'model', 'output_folder'), 
                                     hlpread.read_yaml_key(CONFIGURATION_PATH,'candidate-item-purchase-together', 'item-purchase-together-folder'),
                                     hlpread.read_yaml_key(CONFIGURATION_PATH,'candidate-item-purchase-together', 'item-purchase-together-output')
                                     )

#with open(ITEMS_PAIRED_TOGETHER, 'wb') as fopen:
#    pickle.dump(pairs_items, fopen)

hlwrite.save_object(ITEMS_PAIRED_TOGETHER, pairs_items)

In [6]:
del tran_data
gc.collect()

871

# Display Item Purchased Together #

When customers bought the item in the 1st column below, then those customers also bought the items in the 2nd, 3rd, and 4th column too!


In [ ]:

#Check for top 10 items pairs 
top_pair_item = list(pairs_items.keys())[:10]

for i, k in enumerate( top_pair_item ):
    
    v = pairs_items[k]
    name1 = hlpimage.get_image_path(k)
    name2 = hlpimage.get_image_path(v[0])
    name3 = hlpimage.get_image_path(v[1])
    name4 = hlpimage.get_image_path(v[2])
    
    if exists(name1) & exists(name2) & exists(name3) & exists(name4):
        plt.figure(figsize=(20,5))
        img1 = cv2.imread(name1)[:,:,::-1]
        img2 = cv2.imread(name2)[:,:,::-1]
        img3 = cv2.imread(name3)[:,:,::-1]
        img4 = cv2.imread(name4)[:,:,::-1]
        
        plt.subplot(1,4,1)
        plt.title('When customers buy this',size=18)
        plt.imshow(img1)
        plt.subplot(1,4,2)
        plt.title('They buy this',size=18)
        plt.imshow(img2)
        plt.subplot(1,4,3)
        plt.title('They buy this',size=18)
        plt.imshow(img3)
        plt.subplot(1,4,4)
        plt.title('They buy this',size=18)
        plt.imshow(img4)
        plt.show()